# Debug: Simulate DuckDB Query
Test the exact SQL query that the backend uses to detect imbalances

In [ ]:
# Simulate the SQL query using pandas
# Read the imbalanced Excel file
df_r = pd.read_excel("test_imbalanced.xlsx", sheet_name="RESULTADO")
df_c = pd.read_excel("test_imbalanced.xlsx", sheet_name="CONTABIL")
df_f = pd.read_excel("test_imbalanced.xlsx", sheet_name="FICTICIO")

# Add sheet_name column
df_r["sheetName"] = "RESULTADO"
df_c["sheetName"] = "CONTABIL"
df_f["sheetName"] = "FICTICIO"

# Combine all sheets (simulating read_parquet with all sheets)
df_all = pd.concat([df_r, df_c, df_f], ignore_index=True)

print(f"📊 Total rows across all sheets: {len(df_all)}")
print(f"   RESULTADO: {len(df_r)} rows")
print(f"   CONTABIL: {len(df_c)} rows")
print(f"   FICTICIO: {len(df_f)} rows")
print()

# Check first date column
date_col = date_cols[0]
print(f"🔍 Checking column: {date_col}\n")

# Group by cod, segmentos, sheet_name and sum (this is what SQL does)
grouped = (
    df_all.groupby(["Cod", "Segmentos", "sheetName"])[date_col].sum().reset_index()
)
pivoted = grouped.pivot(
    index=["Cod", "Segmentos"], columns="sheetName", values=date_col
).fillna(0)

print("Pivoted data (first 5 rows):")
print(pivoted.head())
print()

# Calculate differences
pivoted["diff"] = pivoted["RESULTADO"] - (pivoted["CONTABIL"] + pivoted["FICTICIO"])
pivoted["abs_diff"] = pivoted["diff"].abs()

# Filter imbalances
imbalances = pivoted[
    (pivoted["abs_diff"] > 0.01)
    & (
        (pivoted["RESULTADO"].abs() > 0.01)
        | (pivoted["CONTABIL"].abs() > 0.01)
        | (pivoted["FICTICIO"].abs() > 0.01)
    )
]

print(f"Found {len(imbalances)} imbalances")
if len(imbalances) > 0:
    print("\n❌ Top imbalances:")
    print(
        imbalances.nlargest(5, "abs_diff")[
            ["RESULTADO", "CONTABIL", "FICTICIO", "diff"]
        ]
    )

# Check for Duplicate Cod+Segmentos
Verify if there are duplicates within each sheet

In [ ]:
# Check for duplicates
print("🔍 Checking for duplicate Cod+Segmentos combinations:\n")

for sheet_name, df in [("RESULTADO", df_r), ("CONTABIL", df_c), ("FICTICIO", df_f)]:
    duplicates = df.groupby(["Cod", "Segmentos"]).size()
    has_dupes = (duplicates > 1).any()
    print(f"{sheet_name}:")
    print(f"  Total rows: {len(df)}")
    print(f"  Unique Cod+Segmentos: {len(duplicates)}")
    print(f"  Has duplicates: {'❌ YES' if has_dupes else '✅ NO'}")
    if has_dupes:
        print(f"  Max occurrences: {duplicates.max()}")
    print()

# The issue: SUM will add up all rows with same Cod+Segmentos
# If RESULTADO has Cod=100, Seg=X appearing twice, it sums both values
# Let's check what happens with error row 0
test_row = 0
test_cod = df_r.at[test_row, "Cod"]
test_seg = df_r.at[test_row, "Segmentos"]

print(f"\n📌 Checking test row: Cod={test_cod}, Segmentos={test_seg}")
print(f"\nOccurrences in each sheet:")
for sheet_name, df in [("RESULTADO", df_r), ("CONTABIL", df_c), ("FICTICIO", df_f)]:
    matches = df[(df["Cod"] == test_cod) & (df["Segmentos"] == test_seg)]
    print(f"  {sheet_name}: {len(matches)} row(s)")
    if len(matches) > 0 and len(matches) <= 3:
        print(f"    {date_cols[0]} values: {matches[date_cols[0]].tolist()}")

# CRITICAL: Check if sheets have same Cod values
This is the likely issue!

In [ ]:
# THIS IS THE PROBLEM!
# Check if Cod values are the same across sheets

print("🚨 CRITICAL ISSUE FOUND!\n")
print("Checking if sheets have the same Cod values in the same rows:\n")

print(f"Row 0:")
print(f"  RESULTADO Cod: {df_resultado_error.at[0, 'Cod']}")
print(f"  CONTABIL Cod:  {df_contabil.at[0, 'Cod']}")
print(f"  FICTICIO Cod:  {df_ficticio.at[0, 'Cod']}")
print(
    f"  Same? {df_resultado_error.at[0, 'Cod'] == df_contabil.at[0, 'Cod'] == df_ficticio.at[0, 'Cod']}"
)
print()

print(f"Row 0 Segmentos:")
print(f"  RESULTADO: {df_resultado_error.at[0, 'Segmentos']}")
print(f"  CONTABIL:  {df_contabil.at[0, 'Segmentos']}")
print(f"  FICTICIO:  {df_ficticio.at[0, 'Segmentos']}")
print()

# Check if ALL Cod values match
print("Checking if ALL rows have matching Cod values across sheets:")
resultado_cods = set(df_resultado_error["Cod"].tolist())
contabil_cods = set(df_contabil["Cod"].tolist())
ficticio_cods = set(df_ficticio["Cod"].tolist())

print(f"  RESULTADO unique Cods: {len(resultado_cods)}")
print(f"  CONTABIL unique Cods: {len(contabil_cods)}")
print(f"  FICTICIO unique Cods: {len(ficticio_cods)}")
print(
    f"  Common Cods across all 3 sheets: {len(resultado_cods & contabil_cods & ficticio_cods)}"
)
print()

print("❌ THE PROBLEM:")
print("   Each sheet was created from the SAME 'df' which has RANDOM Cod values!")
print("   But the randomization happens EACH TIME, so each sheet might have")
print("   DIFFERENT Cod values in the same row position!")
print()
print("✅ SOLUTION:")
print("   The 'cod' list should be the SAME for all three sheets.")
print("   The balanced/error dataframes are created correctly from cell 4 onwards.")

# Final Verification: Check Exported File Balance
Read the test_imbalanced.xlsx and manually verify the balance

In [ ]:
# Read and verify the actual exported test_imbalanced.xlsx file
print("📂 Reading test_imbalanced.xlsx...\n")

xl_result = pd.read_excel("test_imbalanced.xlsx", sheet_name="RESULTADO")
xl_contabil = pd.read_excel("test_imbalanced.xlsx", sheet_name="CONTABIL")
xl_ficticio = pd.read_excel("test_imbalanced.xlsx", sheet_name="FICTICIO")

# Check a few rows that should have errors
print("Checking rows that should have imbalances (error_rows):\n")

for i, row_idx in enumerate(error_rows[:3]):
    cod = xl_result.at[row_idx, "Cod"]
    seg = xl_result.at[row_idx, "Segmentos"]
    date_col = date_cols[0]

    r_val = xl_result.at[row_idx, date_col]
    c_val = xl_contabil.at[row_idx, date_col]
    f_val = xl_ficticio.at[row_idx, date_col]

    expected = c_val + f_val
    diff = r_val - expected

    print(f"Row {row_idx + 2} (Cod: {cod}, Seg: {seg}, {date_col}):")
    print(f"  RESULTADO: {r_val:>12,.2f}")
    print(f"  CONTABIL:  {c_val:>12,.2f}")
    print(f"  FICTICIO:  {f_val:>12,.2f}")
    print(f"  Expected:  {expected:>12,.2f}")
    print(
        f"  Difference: {diff:>11,.2f} {'❌ IMBALANCED' if abs(diff) > 0.01 else '✅ BALANCED'}"
    )
    print()

print("\n🔍 Summary:")
print(f"If differences are all R$ {imbalance_amount:,.2f}, the file is correct!")
print(f"If differences are R$ 0.00, the export might have used wrong dataframes!")

In [1]:
from locale import setlocale, LC_ALL
from random import choice, randint, uniform, seed
from datetime import date

import pandas as pd

seed(42)
setlocale(LC_ALL, "pt_BR")

'pt_BR'

In [9]:
segments = [
    "Empresas I",
    "Empresas II",
    "Empresas III",
    "Empresas I Massivo",
    # "Empresas 1 Massivo",
    "Select",
    "Select High",
    "Agro Select",
    "Governos",
    "Universidades",
    "Especial",
    "Prospera",
    "MEI",
]

areas = ["Cards", "Loans", "Insurance", "Investments", "Savings", "Payments"]

n_range = 30000
# n_range = 3000
n_range = 1000

date_cols = [date(2025, m, 1).strftime("%b/%y").capitalize() for m in range(1, 13)]

cod = [randint(12, 15000) for _ in range(n_range)]

df = pd.DataFrame(
    {
        "Cod": cod,
        "Itens / Período": [f"Item {c}" for c in cod],
        "Segmentos": [choice(segments) for _ in range(n_range)],
        "File_Paths": [choice(areas) for _ in range(n_range)],
        **{dt: [uniform(-1000, 1000) for _ in range(n_range)] for dt in date_cols},
    }
)

# Create Balanced Data
Create three dataframes where RESULTADO = CONTABIL + FICTICIO for testing balance validation

In [10]:
# Create three dataframes with balanced data
# RESULTADO = CONTABIL + FICTICIO for each row and date column

df_contabil = df.copy()
df_ficticio = df.copy()
df_resultado = df.copy()

# For each date column, make sure the balance equation holds
for date_col in date_cols:
    # Keep CONTABIL as is
    # Keep FICTICIO as is
    # Calculate RESULTADO = CONTABIL + FICTICIO
    df_resultado[date_col] = df_contabil[date_col] + df_ficticio[date_col]

print("✅ Created balanced dataframes where RESULTADO = CONTABIL + FICTICIO")
print(f"Date columns: {date_cols}")
print(f"\nSample verification for first row:")
print(f"  CONTABIL (Jan/25): {df_contabil[date_cols[0]].iloc[0]:.2f}")
print(f"  FICTICIO (Jan/25): {df_ficticio[date_cols[0]].iloc[0]:.2f}")
print(f"  RESULTADO (Jan/25): {df_resultado[date_cols[0]].iloc[0]:.2f}")
print(
    f"  Sum check: {df_contabil[date_cols[0]].iloc[0] + df_ficticio[date_cols[0]].iloc[0]:.2f}"
)

✅ Created balanced dataframes where RESULTADO = CONTABIL + FICTICIO
Date columns: ['Jan/25', 'Fev/25', 'Mar/25', 'Abr/25', 'Mai/25', 'Jun/25', 'Jul/25', 'Ago/25', 'Set/25', 'Out/25', 'Nov/25', 'Dez/25']

Sample verification for first row:
  CONTABIL (Jan/25): 542.75
  FICTICIO (Jan/25): 542.75
  RESULTADO (Jan/25): 1085.51
  Sum check: 1085.51


# Create Imbalanced Data for Testing
Add/subtract values from specific rows in RESULTADO to intentionally break the balance equation

In [11]:
# Create imbalanced data by modifying RESULTADO
df_resultado_error = df_resultado.copy()

# Select some random rows to create imbalances
error_rows = [0, 5, 10, 15, 20]  # First few rows for easy verification

# Option 1: Add a fixed value to specific date columns
# This will break the balance: RESULTADO ≠ CONTABIL + FICTICIO
imbalance_amount = 5000  # R$ 5,000 difference

print("🔴 Creating imbalanced data for testing balance validation\n")

for row_idx in error_rows:
    # Add imbalance to all date columns for these rows
    for date_col in date_cols:
        df_resultado_error.at[row_idx, date_col] += imbalance_amount

    print(f"Row {row_idx + 2} (Excel row):")
    print(f"  Cod: {df_resultado_error.at[row_idx, 'Cod']}")
    print(f"  Segmentos: {df_resultado_error.at[row_idx, 'Segmentos']}")
    print(f"  Added R$ {imbalance_amount:,.2f} to all date columns")
    print(f"  Example ({date_cols[0]}):")
    print(f"    CONTABIL: {df_contabil.at[row_idx, date_cols[0]]:,.2f}")
    print(f"    FICTICIO: {df_ficticio.at[row_idx, date_cols[0]]:,.2f}")
    print(f"    RESULTADO (original): {df_resultado.at[row_idx, date_cols[0]]:,.2f}")
    print(f"    RESULTADO (error): {df_resultado_error.at[row_idx, date_cols[0]]:,.2f}")
    print(
        f"    Expected: {df_contabil.at[row_idx, date_cols[0]] + df_ficticio.at[row_idx, date_cols[0]]:,.2f}"
    )
    print(f"    Difference: R$ {imbalance_amount:,.2f}\n")

🔴 Creating imbalanced data for testing balance validation

Row 2 (Excel row):
  Cod: 12709
  Segmentos: Empresas I Massivo
  Added R$ 5,000.00 to all date columns
  Example (Jan/25):
    CONTABIL: 542.75
    FICTICIO: 542.75
    RESULTADO (original): 1,085.51
    RESULTADO (error): 6,085.51
    Expected: 1,085.51
    Difference: R$ 5,000.00

Row 7 (Excel row):
  Cod: 6662
  Segmentos: Agro Select
  Added R$ 5,000.00 to all date columns
  Example (Jan/25):
    CONTABIL: 826.57
    FICTICIO: 826.57
    RESULTADO (original): 1,653.14
    RESULTADO (error): 6,653.14
    Expected: 1,653.14
    Difference: R$ 5,000.00

Row 12 (Excel row):
  Cod: 7764
  Segmentos: Empresas II
  Added R$ 5,000.00 to all date columns
  Example (Jan/25):
    CONTABIL: -314.12
    FICTICIO: -314.12
    RESULTADO (original): -628.24
    RESULTADO (error): 4,371.76
    Expected: -628.24
    Difference: R$ 5,000.00

Row 17 (Excel row):
  Cod: 5058
  Segmentos: Empresas I
  Added R$ 5,000.00 to all date columns
  Exa

# Alternative: Create Imbalances on Specific Date Columns
Modify only certain months to test date-specific balance validation

In [12]:
# Create imbalances only on specific months
df_resultado_partial_error = df_resultado.copy()

# Select specific date columns to break
error_months = [date_cols[0], date_cols[5], date_cols[11]]  # Jan, Jun, Dec
error_rows_partial = [2, 7, 12]

print("🟡 Creating imbalances on specific months only\n")

for row_idx in error_rows_partial:
    for date_col in error_months:
        # Subtract value from some, add to others
        adjustment = 3000 if date_col == date_cols[0] else -2500
        df_resultado_partial_error.at[row_idx, date_col] += adjustment

        print(f"Row {row_idx + 2} - {date_col}:")
        print(f"  Cod: {df_resultado_partial_error.at[row_idx, 'Cod']}")
        print(f"  Segmentos: {df_resultado_partial_error.at[row_idx, 'Segmentos']}")
        print(f"  Adjustment: R$ {adjustment:,.2f}")
        print(
            f"  New difference: R$ {df_resultado_partial_error.at[row_idx, date_col] - (df_contabil.at[row_idx, date_col] + df_ficticio.at[row_idx, date_col]):,.2f}\n"
        )

🟡 Creating imbalances on specific months only

Row 4 - Jan/25:
  Cod: 14582
  Segmentos: Select High
  Adjustment: R$ 3,000.00
  New difference: R$ 3,000.00

Row 4 - Jun/25:
  Cod: 14582
  Segmentos: Select High
  Adjustment: R$ -2,500.00
  New difference: R$ -2,500.00

Row 4 - Dez/25:
  Cod: 14582
  Segmentos: Select High
  Adjustment: R$ -2,500.00
  New difference: R$ -2,500.00

Row 9 - Jan/25:
  Cod: 7092
  Segmentos: Empresas I
  Adjustment: R$ 3,000.00
  New difference: R$ 3,000.00

Row 9 - Jun/25:
  Cod: 7092
  Segmentos: Empresas I
  Adjustment: R$ -2,500.00
  New difference: R$ -2,500.00

Row 9 - Dez/25:
  Cod: 7092
  Segmentos: Empresas I
  Adjustment: R$ -2,500.00
  New difference: R$ -2,500.00

Row 14 - Jan/25:
  Cod: 5480
  Segmentos: Empresas III
  Adjustment: R$ 3,000.00
  New difference: R$ 3,000.00

Row 14 - Jun/25:
  Cod: 5480
  Segmentos: Empresas III
  Adjustment: R$ -2,500.00
  New difference: R$ -2,500.00

Row 14 - Dez/25:
  Cod: 5480
  Segmentos: Empresas III
  Ad

# Export Test Files
Create both balanced and imbalanced Excel files for testing

In [13]:
# # Export balanced file (should PASS validation)
# with pd.ExcelWriter("test_balanced.xlsx") as writer:
#     df_resultado.to_excel(writer, index=False, sheet_name="RESULTADO")
#     df_contabil.to_excel(writer, index=False, sheet_name="CONTABIL")
#     df_ficticio.to_excel(writer, index=False, sheet_name="FICTICIO")
#     df.to_excel(writer, index=False, sheet_name="SALDO_MEDIO")
#     df.to_excel(writer, index=False, sheet_name="SALDO_PONTA")

# print("✅ Created test_balanced.xlsx - RESULTADO = CONTABIL + FICTICIO (should PASS)")

# Export imbalanced file (should FAIL validation with clear error)
with pd.ExcelWriter("test_imbalanced.xlsx") as writer:
    df_resultado_error.to_excel(writer, index=False, sheet_name="RESULTADO")
    df_contabil.to_excel(writer, index=False, sheet_name="CONTABIL")
    df_ficticio.to_excel(writer, index=False, sheet_name="FICTICIO")
    df.to_excel(writer, index=False, sheet_name="SALDO_MEDIO")
    df.to_excel(writer, index=False, sheet_name="SALDO_PONTA")

print(
    "🔴 Created test_imbalanced.xlsx - RESULTADO has +R$ 5,000 on rows 2,6,11,16,21 (should FAIL)"
)

# Export partial error file (should FAIL on specific months)
with pd.ExcelWriter("test_partial_error.xlsx") as writer:
    df_resultado_partial_error.to_excel(writer, index=False, sheet_name="RESULTADO")
    df_contabil.to_excel(writer, index=False, sheet_name="CONTABIL")
    df_ficticio.to_excel(writer, index=False, sheet_name="FICTICIO")
    df.to_excel(writer, index=False, sheet_name="SALDO_MEDIO")
    df.to_excel(writer, index=False, sheet_name="SALDO_PONTA")

print(
    "🟡 Created test_partial_error.xlsx - Errors only on Jan, Jun, Dec for rows 4,9,14 (should FAIL)"
)

print("\n📁 Files created:")
print("  1. test_balanced.xlsx → Should upload successfully")
print(
    "  2. test_imbalanced.xlsx → Should show balance error with R$ 5.000,00 difference"
)
print("  3. test_partial_error.xlsx → Should show balance error on specific months")

🔴 Created test_imbalanced.xlsx - RESULTADO has +R$ 5,000 on rows 2,6,11,16,21 (should FAIL)
🟡 Created test_partial_error.xlsx - Errors only on Jan, Jun, Dec for rows 4,9,14 (should FAIL)

📁 Files created:
  1. test_balanced.xlsx → Should upload successfully
  2. test_imbalanced.xlsx → Should show balance error with R$ 5.000,00 difference
  3. test_partial_error.xlsx → Should show balance error on specific months
🟡 Created test_partial_error.xlsx - Errors only on Jan, Jun, Dec for rows 4,9,14 (should FAIL)

📁 Files created:
  1. test_balanced.xlsx → Should upload successfully
  2. test_imbalanced.xlsx → Should show balance error with R$ 5.000,00 difference
  3. test_partial_error.xlsx → Should show balance error on specific months


# Debug: Verify Imbalance Data
Check a few rows to ensure the imbalance was actually created

In [ ]:
# Verify the imbalance in the generated dataframes
print("🔍 Verifying imbalances in error dataframe:\n")

for row_idx in error_rows[:3]:  # Check first 3 error rows
    cod_val = df_resultado_error.at[row_idx, "Cod"]
    seg_val = df_resultado_error.at[row_idx, "Segmentos"]

    print(f"Row {row_idx + 2} (Cod: {cod_val}, Segmentos: {seg_val}):")

    for date_col in date_cols[:3]:  # Check first 3 months
        contabil_val = df_contabil.at[row_idx, date_col]
        ficticio_val = df_ficticio.at[row_idx, date_col]
        resultado_val = df_resultado_error.at[row_idx, date_col]

        expected = contabil_val + ficticio_val
        diff = resultado_val - expected

        print(f"  {date_col}:")
        print(f"    CONTABIL: {contabil_val:,.2f}")
        print(f"    FICTICIO: {ficticio_val:,.2f}")
        print(f"    Expected RESULTADO: {expected:,.2f}")
        print(f"    Actual RESULTADO: {resultado_val:,.2f}")
        print(f"    Difference: {diff:,.2f} {'✅' if abs(diff) < 0.01 else '❌'}")
    print()

# Debug: Check Excel File Structure
Verify the exported file has the correct structure

In [ ]:
# Read back the imbalanced file and verify structure
df_check_resultado = pd.read_excel("test_imbalanced.xlsx", sheet_name="RESULTADO")
df_check_contabil = pd.read_excel("test_imbalanced.xlsx", sheet_name="CONTABIL")
df_check_ficticio = pd.read_excel("test_imbalanced.xlsx", sheet_name="FICTICIO")

print("📂 Checking test_imbalanced.xlsx structure:\n")

# Check first error row
row_idx = 0
cod_val = df_check_resultado.at[row_idx, "Cod"]
seg_val = df_check_resultado.at[row_idx, "Segmentos"]

print(f"First row (Cod: {cod_val}, Segmentos: {seg_val}):")
print(f"  Columns: {list(df_check_resultado.columns)[:7]}...")
print()

# Verify balance for first error row on first date column
first_date = date_cols[0]
resultado = df_check_resultado.at[row_idx, first_date]
contabil = df_check_contabil.at[row_idx, first_date]
ficticio = df_check_ficticio.at[row_idx, first_date]
diff = resultado - (contabil + ficticio)

print(f"Balance check for {first_date}:")
print(f"  RESULTADO: {resultado:,.2f}")
print(f"  CONTABIL:  {contabil:,.2f}")
print(f"  FICTICIO:  {ficticio:,.2f}")
print(f"  Sum (C+F): {contabil + ficticio:,.2f}")
print(f"  Difference: {diff:,.2f}")
print(f"  Status: {'❌ IMBALANCED' if abs(diff) > 0.01 else '✅ BALANCED'}")

In [ ]:
# Quick test file with random data (not balanced)
# This will likely have balance errors since values are random
with pd.ExcelWriter("test_random.xlsx") as writer:
    df.to_excel(writer, index=False, sheet_name="RESULTADO")
    df.to_excel(writer, index=False, sheet_name="CONTABIL")
    df.to_excel(writer, index=False, sheet_name="FICTICIO")
    df.to_excel(writer, index=False, sheet_name="SALDO_MEDIO")
    df.to_excel(writer, index=False, sheet_name="SALDO_PONTA")

print("⚠️  Created test_random.xlsx - Random values (likely to have balance errors)")

In [ ]:
import pandas as pd

In [4]:
# all pages under -> 14kB initial load